In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from tqdm import tqdm
from urllib.parse import quote
from folium import *
from folium.plugins import MarkerCluster
import folium
import json

### 1. 데이터를 가져오기

In [2]:
path = "c:/python/chromedriver.exe"
driver = webdriver.Chrome()
url = 'https://hotels.naver.com/item/photo?hotelFileName=hotel%3AStaz_Hotel_Jeju_Robero&adultCnt=2&includeTax=false'
driver.get(url)
time.sleep(1)

driver.find_element(By.XPATH, r'//*[@id="__next"]/div/div/div[2]/div/a[5]').click()
#driver.find_element(By.XPATH, r'//*[@id="__next"]/div/div[1]/div[3]/div/a[5]').click()
#driver.find_element(By.XPATH, r'//*[@id="__next"]/div/div[1]/div[2]/div/a[4]').click()

### 2. 스크롤 내리기

In [3]:
while True:
    bh = driver.execute_script("return document.body.scrollHeight")
    print(bh)
    time.sleep(4)
    driver.execute_script("window.scrollTo(0,document.body.scrollHeight)")
    time.sleep(2)
    ah = driver.execute_script("return document.body.scrollHeight")
    if ah == bh:
        break
    bh = ah

2857
6134


### 2. 원하는 데이터 찾기

In [4]:
soup = BeautifulSoup(driver.page_source, 'html.parser')

In [5]:
lis = soup.select_one(".common_ReviewList__4cL43").find_all('li')
len(lis)

30

In [6]:
li = lis[0]

### 3. 반복문으로 데이터 가져오기

In [7]:
line = []
for li in lis:
    title = li.find(attrs={'class':'common_title_txt__Kn91_'}).get_text()
    score = li.find('b', attrs={'class':'common_star_txt__3f1Yt'}).get_text()
    try : 
        p_review=li.find('p',attrs={"class":"common_booking_txt___dAYT common_as_positive__8PlLV"}).get_text()
    except:
        p_review = "없음"
    try : 
        n_review = li.find('p',attrs={"class":"common_booking_txt___dAYT common_as_negative__LjbBo"}).get_text()
    except:
        n_review = "없음"  
    try : 
        o_review = li.find('p',attrs={"class":"common_txt__2Xcex"}).get_text()
    except:
        o_review = "없음"  
    line.append({'title':title , 'score': score , 'p_review':p_review , 'n_review':n_review,'o_review':o_review  })

driver.close()

### 4. DF 생성  및 저장

In [8]:
# 호텔명
name = soup.select_one('.common_name__TacRt').get_text()
print(name)
# name = soup.select_one('.common_name__TacRt > strong > div' )
# len(lis)

스타즈 호텔 제주 로베로점


In [9]:
df= pd.DataFrame(line)
df.head()

,title,score,p_review,n_review,o_review
0,"""매우 좋음""",8,없음,없음,없음
1,"""강력 추천""",10,없음,없음,없음
2,"""좋음""",7,시장과 가깝고 직원분들이 아주 친절하셨어요\n아기와 있어 카펫 없는 방을 부탁드렸더...,카펫이었던 바닥이 더러웠고 화장실 문의 경첩이 매끄럽게 마무리되어있지않아 문 열고 ...,없음
3,"""매우 좋음""",8,없음,없음,없음
4,"""매우 좋음""",8,없음,없음,없음


In [10]:
df['name']=name
df.head()

,title,score,p_review,n_review,o_review,name
0,"""매우 좋음""",8,없음,없음,없음,스타즈 호텔 제주 로베로점
1,"""강력 추천""",10,없음,없음,없음,스타즈 호텔 제주 로베로점
2,"""좋음""",7,시장과 가깝고 직원분들이 아주 친절하셨어요\n아기와 있어 카펫 없는 방을 부탁드렸더...,카펫이었던 바닥이 더러웠고 화장실 문의 경첩이 매끄럽게 마무리되어있지않아 문 열고 ...,없음,스타즈 호텔 제주 로베로점
3,"""매우 좋음""",8,없음,없음,없음,스타즈 호텔 제주 로베로점
4,"""매우 좋음""",8,없음,없음,없음,스타즈 호텔 제주 로베로점


In [11]:
df.to_csv('c:/프로젝트/Hotel Review/Hotel_review_.csv',encoding="utf-8-sig",index=False)